# Using Sherlock out-of-the-box
This notebook shows how to predict a semantic type for a given table column.
The steps are basically:
- Download files for word embedding and paragraph vector feature extraction (downloads only once) and initialize feature extraction models.
- Extract features from table columns.
- Initialize Sherlock.
- Make a prediction for the feature representation of the column.

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

In [2]:
# %env PYTHONHASHSEED

## Initialize feature extraction models

In [3]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:04.735138 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:02.510114 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.140245 seconds.


[nltk_data] Downloading package punkt to /Users/mmargaret/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mmargaret/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Extract features

In [4]:
semantics_df = pd.DataFrame(columns=['id', 'semantics'])
semantics_dict = {} # {'id':,'semantics':}

In [5]:
import os
_ = os.listdir('/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/data/data_search_e_data_csv/')

In [6]:
file_list = [id for id in _ if '.csv' in id]
len(file_list)

2919

In [7]:
error_list=[]

def extractIDSemanticsWithColumnNames(filename):
    
    IDSemanticsColumns = {'data_filename':filename, 'colSemantics': [], 'colNames':[]}
    try:
        # read files
        a_doc = pd.read_csv('/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/data/data_search_e_data_csv/' 
                            + filename, engine='python')
        a_doc = a_doc.astype(str) #only non-numeric object to str (sherlock required) = .select_dtypes(include=[object])
        data = pd.Series(a_doc.transpose().values.tolist(),name="values") #format it to list of values by columns

        # sherlock extract features
        extract_features("../temporary.csv",data)
        feature_vectors = pd.read_csv("../temporary.csv", dtype=np.float32)

        # sherlock init and predict with pre-trained model
        model = SherlockModel();
        model.initialize_model_from_json(with_weights=True, model_id="sherlock");
        predicted_labels = model.predict(feature_vectors, "sherlock")

        # f_name = filename.split('.', 1)[0] #extract id from filename

        # return dictionary with id: id of the doc, list of the columns' semantics, list of the columns' names
        IDSemanticsColumns = {'data_filename':filename, 'colSemantics': list(predicted_labels), 'colNames':list(a_doc.columns)}
    
    except:
        print('Unable to extract: {}'.format(filename))
        global error_list
        error_list += [filename]
        
    return IDSemanticsColumns

In [8]:
# TEST function
print(file_list[0])
print(extractIDSemanticsWithColumnNames(file_list[0]))
error_list


d2d297eb8b86aa49d40226d4efdb1729515655656dece7bd80b01c6ddabb2b83.text.csv
Unable to extract: d2d297eb8b86aa49d40226d4efdb1729515655656dece7bd80b01c6ddabb2b83.text.csv
{'data_filename': 'd2d297eb8b86aa49d40226d4efdb1729515655656dece7bd80b01c6ddabb2b83.text.csv', 'colSemantics': [], 'colNames': []}


['d2d297eb8b86aa49d40226d4efdb1729515655656dece7bd80b01c6ddabb2b83.text.csv']

In [ ]:
enrich_list = []

for i in range(0, len(file_list)):
    
    enrich_list += [extractIDSemanticsWithColumnNames(file_list[i])]
    if i%25==0:
        pd.DataFrame(enrich_list, 
             columns=['data_filename', 'colSemantics', 'colNames']).to_csv('/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/output/' 
                                                                            +'enriched_part.csv', index=False)
error_list

Extracting Features:   0%|                               | 0/25 [00:00<?, ?it/s]

Unable to extract: d2d297eb8b86aa49d40226d4efdb1729515655656dece7bd80b01c6ddabb2b83.text.csv


Extracting Features:  12%|██▊                    | 3/25 [00:00<00:04,  4.53it/s]

Exporting 1588 column features


Extracting Features: 100%|██████████████████████| 25/25 [00:01<00:00, 15.83it/s]
W0531 19:43:19.632974 4553154048 deprecation.py:506] From /Users/mmargaret/opt/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0531 19:43:19.634391 4553154048 deprecation.py:506] From /Users/mmargaret/opt/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0531 19:43:19.637321 4553154048 deprecation.py:506] From /Users/mmargaret/opt/anaconda3/e

...

Extracting Features: 100%|███████████████████████| 3/3 [00:00<00:00, 138.20it/s]

Unable to extract: 33a29a7736867b93e5d6b53ff13d0ecbea274f42faeeca1c40a4d3c3ca5daf50.text.csv
Unable to extract: 131996a5027d1809789f1b20749c40d158883a4d4f242e568fbc1aa9ae2deffe.text.csv
Exporting 1588 column features



Extracting Features:  14%|███▎                   | 3/21 [00:00<00:00, 21.95it/s]

Unable to extract: eb93a7285703e877412418ea5108d3bc8aebd085c16cdecde9d30ef04b513286.text.csv
Exporting 1588 column features


Extracting Features: 100%|██████████████████████| 21/21 [00:00<00:00, 24.82it/s]


In [ ]:
pd.DataFrame(enrich_list, 
             columns=['data_filename', 'colSemantics', 'colNames']).to_csv('/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/output/' 
                                                                            +'enriched_all.csv', index=False)


In [ ]:
error_list

# TEST in interactive mode

In [9]:
filename = '8b73c4243284ca0c84c45401f2e5a008d65f291466f4661483b432741c0a1836.text.csv'
a_doc = pd.read_csv('/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/data/' + filename)

a_doc = a_doc.select_dtypes(include=[object]).astype(str) #
a_doc.columns


/Users/mmargaret/opt/anaconda3/envs/py37/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


FileNotFoundError: [Errno 2] No such file or directory: '/Users/mmargaret/Documents/[UVA] Thesis/sherlock-project/data/8b73c4243284ca0c84c45401f2e5a008d65f291466f4661483b432741c0a1836.text.csv'

In [4]:
data = pd.Series(
    a_doc.transpose().values.tolist(),
    name="values"
)
data

0    [FFSU, FFSU, FFSU, FFSU, FFSU, FFSU, FFSU, FFS...
1    [XX, XX, XX, XX, XX, XX, XX, XX, XX, XX, XX, X...
2    [PEDIALYTE, PEDIALYTE, ALPRAZOLAM, PRENATAL P,...
3    [4/1, 1/1, 4/1, 4/1, 4/1, 1/1, 1/1, 1/1, 4/1, ...
4    [04/01/2004, 01/01/2004, 04/01/2004, 04/01/200...
5    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
Name: values, dtype: object

In [5]:
extract_features(
    "../temporary.csv",
    data
)
feature_vectors = pd.read_csv("../temporary.csv", dtype=np.float32)

Extracting Features:   0%|                                | 0/6 [00:00<?, ?it/s]

Exporting 1588 column features


Extracting Features: 100%|████████████████████████| 6/6 [00:00<00:00, 11.88it/s]


In [6]:
feature_vectors

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.000095,-0.000920,-0.000721,0.001115,0.000341,0.000725,-0.001038,0.001035,0.000030,-0.000311
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.775455,0.145490,-0.119590,0.208576,-0.846167,-0.376466,-0.025163,-0.637691,0.087774,-0.480676
2,1.0,0.0,0.055,0.071975,0.0,3.0,0.0,55.0,36.710018,5.642683,...,0.546457,-0.071316,0.129075,0.110144,-0.619416,0.449996,0.387853,-0.123300,0.046871,-0.534526
3,1.0,0.0,0.247,0.185991,0.0,1.0,0.0,247.0,-0.623396,1.173288,...,-0.138906,-0.139290,0.016313,0.048243,-0.651826,-0.131029,-0.396145,-0.249700,0.074340,-0.806943
4,1.0,1.0,4.000,0.000000,4.0,4.0,4.0,4000.0,-3.000000,0.000000,...,-0.007073,0.496197,-0.430627,0.539399,0.108929,-0.435804,-0.341008,0.529492,-0.567380,-0.342077
5,1.0,0.0,0.779,0.722159,0.0,3.0,1.0,779.0,-0.049952,0.858064,...,-0.153812,0.345746,-0.562288,-0.068926,-0.093484,-0.301105,-0.297900,0.168808,-0.186550,-0.482700


## Initialize Sherlock

In [7]:
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

W0429 09:13:51.863543 4681580032 deprecation.py:506] From /Users/mmargaret/opt/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0429 09:13:51.864383 4681580032 deprecation.py:506] From /Users/mmargaret/opt/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0429 09:13:51.867066 4681580032 deprecation.py:506] From /Users/mmargaret/opt/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: c

## Predict semantic type for column

In [8]:
predicted_labels = model.predict(feature_vectors, "sherlock")

In [9]:
predicted_labels #predicted columns' name

array(['address', 'state', 'brand', 'country', 'address', 'age'],
      dtype=object)

In [10]:
a_doc.columns #original columns' name

Index(['Utilization Type', 'State', 'Product Name', 'Quarter Begin',
       'Quarter Begin Date', 'Location'],
      dtype='object')

In [11]:
data.iloc[5]

['nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
